# Tacotron 2 Synthesis notebook for your model

## ***Notebook orginized by Justin John***

### Check if your session has a GPU in it (Runtime -> Change runtime type)

#  

5/15/2021 - Fixed model not loding error.

5/10/2021 - Cleaned up.

5/3/2021 - Made a bit more compact.

4/25/2021 - Fixed imports and synthesize, finalized using local file option.

4/24/2021 - Added a local checkpoint option

4/23/2021 - Fixed pre-trained waveglow file by uploading it into mega.nz


In [ ]:
# @markdown # Setting up dependencies, run this cell once. 
%tensorflow_version 1.x
%cd /content/
!apt-get install pv
!apt-get install jq
!wget https://raw.githubusercontent.com/tonikelope/megadown/master/megadown -O megadown.sh
!chmod 755 megadown.sh

import os
from os.path import exists, join, basename, splitext
!pip install gdown
git_repo_url = 'https://github.com/NVIDIA/tacotron2.git'
project_name = splitext(basename(git_repo_url))[0]
if not exists(project_name):
  # clone and install
  !git clone -q --recursive {git_repo_url}
  !cd {project_name}/waveglow && git checkout 2fd4e63
  !pip install -q librosa unidecode
  
import sys
sys.path.append(join(project_name, 'waveglow/'))
sys.path.append(project_name)
import time
import matplotlib
import matplotlib.pylab as plt
import gdown
d = 'https://drive.google.com/uc?id='

In [ ]:
#@title Initialize Tacotron and Waveglow, also run this cell once.
%matplotlib inline
import IPython.display as ipd
import numpy as np
import torch

from hparams import create_hparams
from model import Tacotron2
from layers import TacotronSTFT
from audio_processing import griffin_lim
from text import text_to_sequence
from denoiser import Denoiser

waveglow_pretrained_model = '/content/waveglow.pt'

if not exists(waveglow_pretrained_model):
  print("Installing waveglow pretrained model, may take a bit! (This will only run once)")
  #os.system("/content/megadown.sh https://mega.nz/#!zO5H1KrS!kNwIsmb8EklYHCzVW9-cau-YM_UySthWv88khK4Pydw --o waveglow.pt")
  !/content/megadown.sh https://mega.nz/#!zO5H1KrS!kNwIsmb8EklYHCzVW9-cau-YM_UySthWv88khK4Pydw --o waveglow.pt
  print("Finished!")

graph_width = 900
graph_height = 360
def plot_data(data, figsize=(int(graph_width/100), int(graph_height/100))):
    %matplotlib inline
    fig, axes = plt.subplots(1, len(data), figsize=figsize)
    for i in range(len(data)):
        axes[i].imshow(data[i], aspect='auto', origin='bottom', 
                       interpolation='none', cmap='inferno')
    fig.canvas.draw()
    plt.show()

!gdown --id '1IUxCkgRNsoDCLtMZxYXmIePAKgh0m-aZ'
thisdict = {}
for line in reversed((open('merged.dict.txt', "r").read()).splitlines()):
    thisdict[(line.split(" ",1))[0]] = (line.split(" ",1))[1].strip()
def ARPA(text):
    out = ''
    for word_ in text.split(" "):
        word=word_; end_chars = ''
        while any(elem in word for elem in r"!?,.;") and len(word) > 1:
            if word[-1] == '!': end_chars = '!' + end_chars; word = word[:-1]
            if word[-1] == '?': end_chars = '?' + end_chars; word = word[:-1]
            if word[-1] == ',': end_chars = ',' + end_chars; word = word[:-1]
            if word[-1] == '.': end_chars = '.' + end_chars; word = word[:-1]
            if word[-1] == ';': end_chars = ';' + end_chars; word = word[:-1]
            else: break
        try: word_arpa = thisdict[word.upper()]
        except: word_arpa = ''
        if len(word_arpa)!=0: word = "{" + str(word_arpa) + "}"
        out = (out + " " + word + end_chars).strip()
    if out[-1] != ";": out = out + ";"
    return out

#torch.set_grad_enabled(False)

# initialize Tacotron2 with the pretrained model
hparams = create_hparams()

In [ ]:
%cd /content/
import os
force_download_TT2 = True

#@markdown ![tutorial.png](...)

#@markdown # Enter your model's google drive ID here:

#@markdown Make sure to change restricted to "Anyone with the link" or else it won't be able to download.

model_ID = "1IP4tiIfFdy6ePagEmEgrnbRYfHKebpbi" #@param {type: "string"}

#@markdown ---

#@markdown ### If you are using an uploaded checkpoint instead, check the box and copy the path after uploading it into the files.

using_local_file = False #@param {type:"boolean"}
#@markdown #### Enter the path here:
tacotron2_pretrained_model = "" #@param {type:"string"}

#@markdown ---

if using_local_file == False: 
  tacotron2_pretrained_model = "/content/MLPTTS"

if not exists(tacotron2_pretrained_model) or force_download_TT2 and using_local_file == False:
  gdown.download(d+model_ID, tacotron2_pretrained_model, quiet=False); print("Tacotron2 Model Downloaded")
elif using_local_file == True:
  print("Using local file")

checkpoint_path = tacotron2_pretrained_model
waveglow_path = "/content/waveglow.pt"

# Load Tacotron2 (run this cell every time you change the model)
hparams.sampling_rate = 22050 # Don't change this
hparams.max_decoder_steps = 1000 # How long the audio will be before it cuts off (1000 is about 11 seconds)
hparams.gate_threshold = 0.1 # Model must be 90% sure the clip is over before ending generation (the higher this number is, the more likely that the AI will keep generating until it reaches the Max Decoder Steps)
model = Tacotron2(hparams)
model.load_state_dict(torch.load(tacotron2_pretrained_model)['state_dict'])
_ = model.cuda().eval().half()

# Load WaveGlow
waveglow = torch.load(waveglow_pretrained_model)['model']
waveglow.cuda().eval().half()
for k in waveglow.convinv:
    k.float()
denoiser = Denoiser(waveglow)# Load WaveGlow
waveglow = torch.load(waveglow_pretrained_model)['model']
waveglow.cuda().eval().half()
for k in waveglow.convinv:
    k.float()
denoiser = Denoiser(waveglow)

In [ ]:
#@markdown # **Synthesis**

#@markdown # Enter your desired text here, the graph and audio will show up.

text = "And so he ran across the street to measure the absence of being." #@param {type:"string"}
sigma = 0.8
denoise_strength = 0.324

#@markdown #### Optional if you want automatic ARPA convertion. 

raw_input = True #@param {type:"boolean"}

for i in text.split("\n"):
    if len(i) < 1: continue;
    print(i)
    if raw_input:
        if i[-1] != ";": i=i+";" 
    else: i = ARPA(i)
    print(i)
    with torch.no_grad(): # save VRAM by not including gradients
        sequence = np.array(text_to_sequence(i, ['english_cleaners']))[None, :]
        sequence = torch.autograd.Variable(torch.from_numpy(sequence)).cuda().long()
        mel_outputs, mel_outputs_postnet, _, alignments = model.inference(sequence)
        plot_data((mel_outputs_postnet.float().data.cpu().numpy()[0],
                alignments.float().data.cpu().numpy()[0].T))
        audio = waveglow.infer(mel_outputs_postnet, sigma=sigma); print(""); ipd.display(ipd.Audio(audio[0].data.cpu().numpy(), rate=hparams.sampling_rate))